# TNT Single-Mode: Stress Relaxation
> **Handbook:** This notebook demonstrates the TNT Single-Mode model. For complete mathematical derivations and theoretical background, see [TNT Single-Mode Documentation](../../docs/source/models/tnt/index.rst).


**Estimated Time:** 3-5 minutes

## Protocol: Stress Relaxation

**Stress relaxation** measures the time-dependent stress decay after a step strain is applied and held constant. This protocol directly probes the material's intrinsic relaxation timescales and provides the stress relaxation modulus $G(t)$.

### Physical Context for Transient Networks

In associative polymer networks, stress relaxation arises from two mechanisms:

1. **Bond breakage**: Reversible crosslinks dissociate, allowing chains to relax toward equilibrium
2. **Network reorganization**: New bonds form in stress-free configurations

For TNT Single-Mode with constant breakage rate $\beta = 1/\tau_b$, the model predicts **single-exponential** decay:

$$
G(t) = G \exp(-t/\tau_b)
$$

where:
- $G$: Elastic modulus (plateau modulus at t=0⁺)
- $\tau_b$: Breakage time (characteristic relaxation time)

The exponential form is a signature of **Maxwell-like behavior** — one dominant relaxation mechanism.

### Why TNT Single-Mode for Relaxation?

The Tanaka-Edwards model is ideal for:
- **Pedagogical baseline**: Understanding how $\beta = 1/\tau_b$ leads to exponential decay
- **Single-mode networks**: Materials with one dominant crosslink type (e.g., simple telechelic polymers)
- **Comparison to Maxwell model**: Demonstrating TNT-UCM equivalence

For materials showing **multi-exponential decay** or **power-law relaxation**, see:
- **Sticky Rouse (NB27)**: Hierarchical relaxation with multiple sticker timescales
- **Multi-Species (NB21)**: Multiple bond types with independent lifetimes

> **TNT Relaxation Equations**  
> For the analytical solution and numerical implementation, see:  
> [../../docs/source/models/tnt/tnt_protocols.rst](../../docs/source/models/tnt/tnt_protocols.rst) — Section on "Stress Relaxation"

---

## Learning Objectives

1. Understand stress relaxation in transient network materials
2. Fit TNT single-mode model to relaxation modulus data
3. Extract relaxation time $\tau_b$ from exponential decay
4. Predict relaxation modulus $G(t)$ at different strain amplitudes
5. Compare experimental data to single-exponential theory
6. Perform Bayesian inference for parameter uncertainty

## Prerequisites

- Understanding of transient network theory (Notebook 01)
- Familiarity with linear viscoelasticity
- Knowledge of Maxwell model

## Runtime Estimate

- NLSQ fitting: ~2-5 seconds
- Bayesian inference (demo): ~30-60 seconds
- Total: ~2-3 minutes

## 1. Setup

In [ ]:
import os
import sys
import time

# Google Colab support
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax

import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from rheojax.core.jax_config import safe_import_jax
jax, jnp = safe_import_jax()
from rheojax.core.jax_config import verify_float64
verify_float64()

from rheojax.models.tnt import TNTSingleMode

sys.path.insert(0, os.path.dirname(os.path.abspath("")))
sys.path.insert(0, os.path.join("..", "utils"))
from tnt_tutorial_utils import (
    load_laponite_relaxation,
    compute_fit_quality,
    print_convergence_summary,
    print_parameter_comparison,
    save_tnt_results,
    get_tnt_single_mode_param_names,
)

from utils.plotting_utils import (
    plot_nlsq_fit, display_arviz_diagnostics, plot_posterior_predictive
)

# Residual analysis
residuals = stress - model.predict(gamma_dot, test_mode="flow_curve")
print(f"\nResidual Statistics:")
print(f"  Mean residual = {np.mean(residuals):.4e}")
print(f"  Std residual = {np.std(residuals):.4e}")
print(f"  Max absolute residual = {np.max(np.abs(residuals)):.4e}")


### Bayesian Convergence Diagnostics

When running full Bayesian inference (FAST_MODE=0), monitor these diagnostic metrics to ensure MCMC convergence:

| Metric | Acceptable Range | Interpretation |
|--------|------------------|----------------|
| **R-hat** | < 1.01 | Measures chain convergence; values near 1.0 indicate chains mixed well |
| **ESS (Effective Sample Size)** | > 400 | Number of independent samples; higher is better |
| **Divergences** | < 1% of samples | Indicates numerical instability; should be near zero |
| **BFMI (Bayesian Fraction of Missing Information)** | > 0.3 | Low values suggest reparameterization needed |

**Troubleshooting poor diagnostics:**
- High R-hat (>1.01): Increase `num_warmup` or `num_chains`
- Low ESS (<400): Increase `num_samples` or check for strong correlations
- Many divergences: Increase `target_accept` (default 0.8) or use NLSQ warm-start


## 2. Theory

## Stress Relaxation in Transient Networks

### Physical Mechanism

In a stress relaxation experiment:
1. **Instantaneous strain** γ₀ is applied at t = 0
2. **Strain is held constant** while stress is monitored
3. **Stress decays** as network strands break and reform

### Governing Equation

For a transient network with **constant breakage rate** β = 1/τ_b, the stress relaxation modulus is:

$$
G(t) = G \exp\left(-\frac{t}{\tau_b}\right)
$$

This is identical to the **Maxwell model** — single-exponential decay.

### Relaxation Time

The relaxation time τ_b is the time for stress to decay to 1/e ≈ 37% of its initial value:

$$
G(\tau_b) = G \cdot e^{-1} \approx 0.368 \, G
$$

### Material Analogy: Laponite Gels

**Laponite** is a synthetic clay that forms colloidal gels through electrostatic interactions:
- Disk-shaped nanoparticles (25 nm diameter, 1 nm thickness)
- Negative faces, positive edges → **house of cards** structure
- Electrostatic bonds ≈ **transient crosslinks** in TNT

This makes Laponite an **excellent target** for TNT models.

### Parameters

| Parameter | Symbol | Physical Meaning | Typical Range |
|-----------|--------|------------------|---------------|
| Elastic modulus | $G$ | Network stiffness | 1-1000 Pa |
| Breakage time | $\tau_b$ | Relaxation time | 0.01-100 s |
| Solvent viscosity | $\eta_s$ | Background viscosity | 0.001-1 Pa·s |

## 3. Load Data

In [ ]:
# Load stress relaxation data (Laponite gel)
aging_time = 1800  # seconds
t_data, G_t = load_laponite_relaxation(aging_time=aging_time)

print(f"Data shape: {len(t_data)} points")
print(f"Time range: {t_data.min():.3f} - {t_data.max():.3f} s")
print(f"G(t) range: {G_t.min():.2f} - {G_t.max():.2f} Pa")
print(f"Laponite aging time: {aging_time} s")

In [ ]:
# Plot raw data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Linear scale
ax1.plot(t_data, G_t, 'o', markersize=6, alpha=0.7, label='Data')
ax1.set_xlabel('Time (s)', fontsize=12)
ax1.set_ylabel(r'Relaxation modulus $G(t)$ (Pa)', fontsize=12)
ax1.set_title('Stress Relaxation (Linear)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Log scale
ax2.semilogy(t_data, G_t, 'o', markersize=6, alpha=0.7, label='Data')
ax2.set_xlabel('Time (s)', fontsize=12)
ax2.set_ylabel(r'Relaxation modulus $G(t)$ (Pa)', fontsize=12)
ax2.set_title('Stress Relaxation (Log)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=11)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")

print(f"\nInitial modulus: G(0) ≈ {G_t[0]:.2f} Pa")
print(f"Final modulus: G(t_max) ≈ {G_t[-1]:.2f} Pa")
print(f"Decay ratio: G(t_max)/G(0) = {G_t[-1]/G_t[0]:.3f}")

## 4. NLSQ Fitting

In [ ]:
# Create model and fit
# Note: For relaxation, we need to provide gamma_dot used before relaxation starts
gamma_dot = 10.0  # s^-1 (typical prestrain rate)

model = TNTSingleMode(breakage="constant")

start_time = time.time()
result = model.fit(t_data, G_t, test_mode="relaxation", gamma_dot=gamma_dot, method='scipy')
fit_time = time.time() - start_time

print(f"\nNLSQ fitting completed in {fit_time:.2f} seconds")
print(f"\nFitted Parameters:")
param_names = get_tnt_single_mode_param_names(breakage="constant")
for name in param_names:
    param = model.parameters.get(name)
    print(f"  {name} = {param.value:.4e}")

In [ ]:
# Compute fit quality metrics
metrics = compute_fit_quality(G_t, model.predict(t_data, test_mode="relaxation", gamma_dot=gamma_dot))
print(f"\nFit Quality:")
print(f"  R² = {metrics['R2']:.6f}")
print(f"  RMSE = {metrics['RMSE']:.4e} Pa")
print(f"  NRMSE = {metrics['NRMSE']*100:.2f}%")

In [ ]:
# Plot NLSQ fit with uncertainty band
fig, ax = plot_nlsq_fit(
    t_data, G_t, model, test_mode="relaxation",
    param_names=param_names, log_scale=True,
    xlabel='Time (s)',
    ylabel=r'Relaxation modulus $G(t)$ (Pa)',
    title=f'NLSQ Fit (R\u00b2 = {metrics["R2"]:.4f})',
    gamma_dot=gamma_dot
)
plt.close("all")

### 4.2 Physical Analysis

In [ ]:
time_fine = np.linspace(t_data.min(), t_data.max(), 200)
G_t_pred = model.predict(time_fine, test_mode='relaxation', gamma_dot=gamma_dot)
G_pred = G_t_pred
# Extract fitted parameters
G = model.parameters.get('G').value
tau_b = model.parameters.get('tau_b').value
eta_s = model.parameters.get('eta_s').value

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# 1. Compare with single-exponential fit
ax = axes[0, 0]

# Fit simple exponential: G(t) = G0 * exp(-t/tau)
def single_exp(t, G0, tau):
    return G0 * np.exp(-t / tau)

popt, _ = curve_fit(single_exp, t_data, G_t, p0=[G_t[0], 1.0], maxfev=5000)
G0_fit, tau_fit = popt

ax.semilogy(t_data, G_t, 'o', label='Data', markersize=6, alpha=0.7)
ax.semilogy(time_fine, G_t_pred, '-', label=f'TNT (τ_b = {tau_b:.3f} s)', linewidth=2)
ax.semilogy(time_fine, single_exp(time_fine, G0_fit, tau_fit), '--', 
            label=f'Single exp (τ = {tau_fit:.3f} s)', linewidth=2)
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel(r'Relaxation modulus $G(t)$ (Pa)', fontsize=11)
ax.set_title('Exponential Decay Comparison', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 2. Normalized relaxation
ax = axes[0, 1]
G_normalized = G_t / G_t[0]
G_pred_normalized = G_t_pred / G_t_pred[0]

ax.semilogy(t_data / tau_b, G_normalized, 'o', label='Data', markersize=6, alpha=0.7)
ax.semilogy(time_fine / tau_b, G_pred_normalized, '-', label='TNT Fit', linewidth=2)
ax.axhline(y=np.exp(-1), color='k', linestyle='--', alpha=0.5, label='1/e ≈ 0.368')
ax.axvline(x=1, color='r', linestyle=':', alpha=0.5, label='t = τ_b')
ax.set_xlabel(r'Dimensionless time $t/\tau_b$', fontsize=11)
ax.set_ylabel(r'$G(t)/G(0)$', fontsize=11)
ax.set_title('Normalized Relaxation', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 3. Relaxation spectrum
ax = axes[1, 0]
# For single-mode Maxwell: H(τ) = G·δ(τ - τ_b)
ax.axvline(x=tau_b, color='C1', linewidth=3, label=f'τ_b = {tau_b:.3f} s')
ax.axhline(y=G, color='C0', linestyle='--', alpha=0.5, label=f'G = {G:.2f} Pa')
ax.set_xscale('log')
ax.set_xlabel('Relaxation time (s)', fontsize=11)
ax.set_ylabel('Relaxation strength (Pa)', fontsize=11)
ax.set_title('Discrete Relaxation Spectrum', fontsize=12, fontweight='bold')
ax.set_ylim([0, G * 1.2])
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# 4. Residual analysis
ax = axes[1, 1]
residuals = G_t - model.predict(t_data, test_mode="relaxation")
relative_residuals = residuals / G_t * 100

ax.plot(t_data, relative_residuals, 'o', markersize=6, alpha=0.7)
ax.axhline(y=0, color='k', linestyle='--', alpha=0.5)
ax.fill_between([t_data.min(), t_data.max()], -5, 5, alpha=0.2, color='green')
ax.set_xlabel('Time (s)', fontsize=11)
ax.set_ylabel('Relative residual (%)', fontsize=11)
ax.set_title('Residual Analysis', fontsize=12, fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.close("all")

print(f"\nPhysical Interpretation:")
print(f"  Network modulus: G = {G:.2f} Pa")
print(f"  Relaxation time: τ_b = {tau_b:.4f} s")
print(f"  At t = τ_b: G(τ_b) = {G * np.exp(-1):.2f} Pa (37% of initial)")
print(f"  Agreement with single-exponential fit: Δτ = {abs(tau_b - tau_fit):.4f} s")

## 5. Bayesian Inference with NUTS

In [ ]:
# FAST_MODE: Use reduced MCMC for quick validation
# FAST_MODE controls Bayesian inference (env var FAST_MODE, default=1)
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

# Configuration
NUM_WARMUP = 200
NUM_SAMPLES = 500
NUM_CHAINS = 1

if FAST_MODE:
    print("FAST_MODE: Skipping Bayesian inference (JIT compilation takes >600s)")
    print("To run Bayesian analysis, run with FAST_MODE=0")
    # Create a placeholder result with current NLSQ parameters
    class BayesianResult:
        def __init__(self, model, param_names):
            self.posterior_samples = {name: np.array([model.parameters.get_value(name)] * NUM_SAMPLES) for name in param_names}
    bayesian_result = BayesianResult(model, param_names)
    bayes_time = 0.0
else:
    print(f"Running NUTS with {NUM_CHAINS} chain(s)...")
    print(f"Warmup: {NUM_WARMUP} samples, Sampling: {NUM_SAMPLES} samples")
    
    start_time = time.time()
    bayesian_result = model.fit_bayesian(
        t_data, G_t,
        test_mode='relaxation',
        gamma_dot=gamma_dot,
        num_warmup=NUM_WARMUP,
        num_samples=NUM_SAMPLES,
        num_chains=NUM_CHAINS,
        seed=42
    )
    bayes_time = time.time() - start_time
    
    print(f"\nBayesian inference completed in {bayes_time:.1f} seconds")


In [ ]:
# Skip convergence diagnostics in CI mode
if not FAST_MODE:
    print_convergence_summary(bayesian_result, param_names)
else:
    print("FAST_MODE: Skipping convergence diagnostics")


## ArviZ Diagnostics

In [ ]:
# ArviZ diagnostics (trace, pair, forest, energy, autocorrelation, rank)
if not FAST_MODE and hasattr(bayesian_result, 'to_inference_data'):
    display_arviz_diagnostics(bayesian_result, param_names, fast_mode=FAST_MODE)
else:
    print("FAST_MODE: Skipping ArviZ diagnostics")

In [ ]:
# Posterior predictive check
if not FAST_MODE and hasattr(bayesian_result, 'posterior_samples'):
    fig, ax = plot_posterior_predictive(
        t_data,
        G_t,
        model, bayesian_result, test_mode="relaxation",
        param_names=param_names, log_scale=True,
        xlabel=r'Time (s)',
        ylabel=r'Relaxation modulus $G(t)$ (Pa)', gamma_dot=gamma_dot
    )
    plt.close("all")
else:
    print("FAST_MODE: Skipping posterior predictive")

In [ ]:
# Parameter comparison table
print_parameter_comparison(model, bayesian_result.posterior_samples, param_names)

## 6. Physical Interpretation

### Single-Exponential Relaxation

The TNT constant breakage model predicts **single-exponential relaxation**:

$$
G(t) = G \exp\left(-\frac{t}{\tau_b}\right)
$$

This is identical to the **Maxwell model** — the simplest viscoelastic model with one relaxation time.

### Physical Meaning of τ_b

The breakage time τ_b represents:
- **Average crosslink lifetime** in the transient network
- **Time scale** for stress relaxation
- At t = τ_b, stress has decayed to **37% (1/e)** of initial value

### Laponite Gel Structure

Laponite forms a **house of cards** structure:
- Disk-shaped particles with negative faces and positive edges
- Electrostatic attractions create transient crosslinks
- **Aging** increases crosslink density → longer τ_b

The fitted τ_b = {tau_b:.4f} s represents the lifetime of these electrostatic bonds.

### Relaxation vs. Retardation

Two types of linear viscoelastic experiments:
1. **Stress relaxation**: G(t) at constant strain → measures relaxation times
2. **Creep compliance**: J(t) at constant stress → measures retardation times

For Maxwell model: τ_relaxation = τ_retardation = τ_b

### Multi-Mode Generalization

Real materials often show **multi-exponential relaxation**:

$$
G(t) = \sum_{i=1}^N G_i \exp\left(-\frac{t}{\tau_{b,i}}\right)
$$

This requires TNT multi-mode models (see advanced tutorials).

## 7. Save Results

In [ ]:
# Save results
save_tnt_results(model, bayesian_result, "single_mode", "relaxation", param_names)
print("\nResults saved successfully!")

## Key Takeaways

1. **Single-exponential decay**: G(t) = G·exp(-t/τ_b) for constant breakage
2. **Relaxation time**: τ_b is average crosslink lifetime
3. **1/e criterion**: At t = τ_b, stress decays to 37% of initial value
4. **Laponite gels**: Excellent target for TNT (electrostatic bonds = transient crosslinks)
5. **Maxwell equivalence**: TNT constant breakage = Maxwell model
6. **Bayesian inference**: Quantifies uncertainty in relaxation time

## Next Steps

- **Notebook 04**: Creep compliance for retardation time
- **Notebook 05**: SAOS for frequency-dependent moduli G'(ω), G''(ω)
- **Notebook 06**: LAOS for nonlinear harmonics

## Further Reading

### TNT Documentation

- **[TNT Model Family Overview](../../docs/source/models/tnt/index.rst)**: Complete guide to all 5 TNT models
- **[TNT Protocols Reference](../../docs/source/models/tnt/tnt_protocols.rst)**: Mathematical framework for all protocols
- **[TNT Knowledge Extraction](../../docs/source/models/tnt/tnt_knowledge_extraction.rst)**: Guide for interpreting fitted parameters

### Related Notebooks

Explore other protocols in this model family and compare with advanced TNT models.


### Key References

1. **Tanaka, F., & Edwards, S. F.** (1992). Viscoelastic properties of physically crosslinked networks. 1. Transient network theory. *Macromolecules*, 25(5), 1516-1523. [DOI: 10.1021/ma00031a024](https://doi.org/10.1021/ma00031a024)
   - **Original TNT framework**: Conformation tensor dynamics for reversible networks

2. **Green, M. S., & Tobolsky, A. V.** (1946). A new approach to the theory of relaxing polymeric media. *Journal of Chemical Physics*, 14(2), 80-92. [DOI: 10.1063/1.1724109](https://doi.org/10.1063/1.1724109)
   - **Transient network foundation**: Network strand creation and breakage kinetics

3. **Yamamoto, M.** (1956). The visco-elastic properties of network structure I. General formalism. *Journal of the Physical Society of Japan*, 11(4), 413-421. [DOI: 10.1143/JPSJ.11.413](https://doi.org/10.1143/JPSJ.11.413)
   - **Network viscoelasticity theory**: Mathematical formulation of temporary networks

4. **Bell, G. I.** (1978). Models for the specific adhesion of cells to cells. *Science*, 200(4342), 618-627. [DOI: 10.1126/science.347575](https://doi.org/10.1126/science.347575)
   - **Bell breakage model**: Stress-dependent bond dissociation kinetics

5. **Sprakel, J., Spruijt, E., Cohen Stuart, M. A., van der Gucht, J., & Besseling, N. A. M.** (2008). Universal route to a state of pure shear flow. *Physical Review Letters*, 101(24), 248304. [DOI: 10.1103/PhysRevLett.101.248304](https://doi.org/10.1103/PhysRevLett.101.248304)
   - **TNT experimental validation**: Flow curve measurements and rheological signatures
